# Module 10 Application

## Challenge: Crypto Clustering

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "4" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "6" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "4" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "6" # export NUMEXPR_NUM_THREADS=6

os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 


In [3]:
# Import required libraries and dependencies
import hvplot.pandas

import pandas as pd

from pathlib               import Path
from sklearn.cluster       import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [4]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("data/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [5]:
# Generate summary statistics
df_market_data.describe()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [6]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)


:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [7]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)


In [8]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data

In this section, you will use the elbow method to find the best value for `k`.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [9]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1,11))


In [10]:
# Create an empy list to store the inertia values
inertia = []


In [11]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)


In [12]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k":       k,
    "inertia": inertia,
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data = pd.DataFrame(elbow_data)


In [13]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_fig = df_elbow_data.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k,
)

elbow_fig


:Curve   [k]   (inertia)

#### Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer:** The best number for k is its value where the curve flattens its angle. This seems to occur at **k = 4**.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In this section, you will use the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the original data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [14]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)


In [15]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)


KMeans(n_clusters=4)

In [16]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model.predict(df_market_data_scaled)

# View the resulting array of cluster values.
crypto_clusters

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0])

In [17]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled["SegmentedClusters"] = crypto_clusters

# Display sample data
df_market_data_scaled.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,SegmentedClusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,3


In [18]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_fig = df_market_data_scaled.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="SegmentedClusters",
    hover_cols="all",
    legend="bottom",
    height=400,
)

scatter_fig


:NdOverlay   [SegmentedClusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,SegmentedClusters)

---

### Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [19]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [20]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the Array. 
crypto_pca_data[:5]


array([[ 2.05913887, -0.53668446, -0.43823774],
       [ 1.95640089, -0.41082149, -1.0169655 ],
       [-0.99014153, -0.45422783,  0.69002131],
       [-0.9731886 , -0.49464992,  0.55123402],
       [ 3.33036692, -1.02231141, -0.50134143]])

In [21]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_


array([0.38939231, 0.29166039, 0.20810061])

In [22]:
sum(pca.explained_variance_ratio_) * 100

88.91533061505712

#### Answer the following question: What is the total explained variance of the three principal components?

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 88.99%.

In [23]:
# Create a new DataFrame with the PCA data.
df_crypto_pca = pd.DataFrame(
    crypto_pca_data,
    columns=["PC1", "PC2", "PC3"],
)

# Copy the crypto names from the original data
df_crypto_pca["CoinId"] = df_market_data_scaled.index

# Set the coinid column as index
df_crypto_pca.set_index("CoinId", inplace=True)

# Display sample data
df_crypto_pca.head()


,PC1,PC2,PC3
CoinId,,,
bitcoin,2.059139,-0.536684,-0.438238
ethereum,1.956401,-0.410821,-1.016966
tether,-0.990142,-0.454228,0.690021
ripple,-0.973189,-0.494650,0.551234
bitcoin-cash,3.330367,-1.022311,-0.501341


---

### Find the Best Value for k Using the PCA Data

In this section, you will use the elbow method to find the best value for `k` using the PCA data.

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [24]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))


In [25]:
# Create an empy list to store the inertia values
inertia = []


In [26]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_crypto_pca)
    inertia.append(model.inertia_)
    
inertia[:5]


[325.408423385105,
 195.3099505575719,
 108.66475812556443,
 39.88332402229195,
 29.567505320080716]

In [27]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k":       k,
    "inertia": inertia,
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data = pd.DataFrame(elbow_data)
df_elbow_data.head()


,k,inertia
0,1,325.408423
1,2,195.309951
2,3,108.664758
3,4,39.883324
4,5,29.567505


In [28]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_fig = df_elbow_data.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k,
)

pca_elbow_fig


:Curve   [k]   (inertia)

#### Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?
- **Question:** What is the best value for `k` when using the PCA data?
 - **Answer:** The best number for k is its value where the curve flattens its angle. This seems to occur at **k = 4**.<br><br>
- **Question:** Does it differ from the best k value found using the original data?
 - **Answer:** It does not differ from when using the original data.

---

### Cluster Cryptocurrencies with K-means Using the PCA Data

In this section, you will use the PCA data and the K-Means algorithm with the best value for `k` found in the previous section to cluster the cryptocurrencies according to the principal components.

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [29]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)


In [30]:
# Fit the K-Means model using the PCA data
model.fit(df_crypto_pca)


KMeans(n_clusters=4)

In [31]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_pca_clusters = model.predict(df_crypto_pca)

# View the resulting array of cluster values.
crypto_pca_clusters


array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 1, 3, 1, 1, 1, 1])

In [32]:
# Add a new column to the DataFrame with the predicted clusters
df_crypto_pca["PCA_SegmentedClusters"] = crypto_pca_clusters

# Display sample data
df_crypto_pca


,PC1,PC2,PC3,PCA_SegmentedClusters
CoinId,,,,
bitcoin,2.059139,-0.536684,-0.438238,0
ethereum,1.956401,-0.410821,-1.016966,0
tether,-0.990142,-0.454228,0.690021,1
ripple,-0.973189,-0.494650,0.551234,1
bitcoin-cash,3.330367,-1.022311,-0.501341,0
binancecoin,2.527411,-0.422464,-0.287137,0
chainlink,2.603960,-0.387534,-2.182795,0
cardano,2.305615,-0.280835,-1.175874,0
litecoin,1.829981,-0.605545,-0.800958,0


In [33]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_scatter_fig = df_crypto_pca.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="PCA_SegmentedClusters",
    hover_cols="all",
    legend="bottom",
    height=400
)

pca_scatter_fig


:NdOverlay   [PCA_SegmentedClusters]
   :Scatter   [PC1]   (PC2,CoinId,PC3,PCA_SegmentedClusters)

---

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

1. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

> **Rewind:** Back in Lesson 3 of Module 6, you learned how to create composite plots. You can look at that lesson to review how to make these plots; also, you can check [the hvPlot documentation](https://holoviz.org/tutorial/Composing_Plots.html).

In [34]:
# Composite plot to contrast the Elbow curves
elbow_fig.opts(    width=400, title="Elbow Curve") + \
pca_elbow_fig.opts(width=400, title="PCA Elbow Curve")


:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [35]:
# Compoosite plot to contrast the clusters
scatter_fig.opts(    width=400, title="KMeans Clustering") + \
pca_scatter_fig.opts(width=400, title="KMeans Cluster with PCA")

:Layout
   .NdOverlay.I  :NdOverlay   [SegmentedClusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,SegmentedClusters)
   .NdOverlay.II :NdOverlay   [PCA_SegmentedClusters]
      :Scatter   [PC1]   (PC2,CoinId,PC3,PCA_SegmentedClusters)

Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?<br>

 - **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?
  - **Answer:** After using fewer features to cluster the data using K-Means, the principal component analysis yields much more definitive and pronounced results.